In [5]:
# !pip install langchain_google_genai
# !pip install docx2txt pypdf unstructured
# !pip install langchain_community
# !pip install --upgrade langchain-core
# !pip install langchain_chroma
# !pip install sentence_transformers

In [6]:
import langchain

In [7]:
print(langchain.__version__)

0.3.25


In [8]:
import google.generativeai as genai

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY"
os.environ["LANGCHAIN_PROJECT"] = "default"


In [11]:
# from langchain_openai import ChatOpenAI
# from openai import RateLimitError
# import time
# import os

# # Ensure OpenAI API key is set
# # os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY" # Make sure this is set correctly

# llm = ChatOpenAI(model='gpt-4o-mini')

# try:
#     llm_response = llm.invoke("Tell me a joke")
#     print(llm_response)
# except RateLimitError as e:
#     print(f"Rate limit exceeded: {e}")
#     print("Please check your OpenAI plan and billing details.")
#     # You could add a retry mechanism here with increasing delays
#     # For example: time.sleep(60) and then retry the invoke call.
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")

In [ ]:
genai.configure(api_key="GOOGLE_API_KEY")

# List available models to see the correct names (optional, but good for debugging)
#print("Available Gemini models supporting generateContent:")
#for m in genai.list_models():
#  if 'generateContent' in m.supported_generation_methods:
#    print(m.name)

# Use the correct model name for Gemini Pro, which is typically 'gemini-1.0-pro'
response=''
try:
    model = genai.GenerativeModel("gemini-2.0-flash") # Changed model name to gemini-1.0-pro
    response = model.generate_content("Tell me a joke")
    print(response.text)
except Exception as e:
    print(f"An error occurred with the Gemini model: {e}")

Why don't scientists trust atoms?

Because they make up everything!



In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(response.text)

"Why don't scientists trust atoms?\n\nBecause they make up everything!\n"

# **Simple Chain**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the Gemini model as a LangChain Runnable
# Use the correct model name, e.g., "gemini-1.0-pro" or "gemma-7b-it"
# Ensure you have the GOOGLE_API_KEY environment variable set or pass api_key directly
google_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key="GOOGLE_API_KEY")

# Create the chain by piping the LangChain Google model with the output parser
# Each time this chain is called, all these processes get
chain = google_llm | output_parser

# Invoke the chain with a prompt
print(chain.invoke("Tell me a joke"))
# %%

Why don't scientists trust atoms?

Because they make up everything!


# **Structured Output**

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
  phone_model: str = Field(description="Name and model of the phone")
  rating: float = Field(description="Overall rating out of 5")
  pros: List[str] = Field(description="List of Positive aspects")
  cons: List[str] = Field(description="List of Negative aspects")
  summary: str = Field(description="Summary of the review")

google_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key="GOOGLE_API_KEY")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous, colors pop like crazy. Camera's insane too, ezpeecially at night - my Insta game's never been stronger.
Battery life's solid, lasts me all day, no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also still getting used to the new button layout, keep hitting  Bixby by mistake

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = google_llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output

MobileReview(phone_model='Galaxy S21', rating=4.0, pros=['Gorgeous screen', 'Insane camera', 'Solid battery life'], cons=['Pricey', 'No charger', 'Accidental Bixby presses'], summary="Great phone, but a few annoying quirks keep it from being perfect. If you're due for an upgrade, definitely worth checking out!")

In [16]:
output.pros

['Gorgeous screen', 'Insane camera', 'Solid battery life']

# **Prompt Template in Langchain**

In [17]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me a short hoke about {topic}")
prompt.invoke({"topic" : "programming"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short hoke about programming', additional_kwargs={}, response_metadata={})])

In [18]:
chain = prompt | google_llm | output_parser
chain.invoke({"topic":"programmer"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [ ]:
# Putting everything together
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define prompt
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# Initialise the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key="GOOGLE_API_KEY")

# Create output parser
output_parser = StrOutputParser()

# Compose chain
chain = prompt | llm | output_parser

# Use chain
result = chain.invoke({"topic":"programming"})
print(result)

Why do programmers prefer dark mode?

Because light attracts bugs!


# **LLM Messages**

In [20]:
from langchain_core.messages import HumanMessage, SystemMessage

#  the system message serves as a guide for the model on how to use the retrieved documents to answer a user's query.
system_message = SystemMessage(content="You are a helpful assistant that tells jokes")

human_message = HumanMessage(content='Tell me about programming')

google_llm.invoke([system_message, human_message])

AIMessage(content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs! ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--eb2fccf7-291c-41c2-9781-97ea6b4febf0-0', usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [21]:
# Putting system and human messages in to a template

template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes"),
    ("human", "Tell me about {topic}")
])

# Setting the prompt in the placeholder value
prompt_value = template.invoke({
    "topic":"programming"
})

# Displaying the prompt generated
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that tells jokes', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about programming', additional_kwargs={}, response_metadata={})])

In [22]:
# Invoking LLM with the predefined chain
google_llm.invoke(prompt_value)

AIMessage(content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs! ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b62e34cc-6345-445c-bda3-915d6e876161-0', usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [24]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

# Splitting text from sources into Docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len,
    add_start_index = True
)

docs_loader = PyPDFLoader('/content/annualreport-2023.pdf')
docs = docs_loader.load()
print(docs)
splits = text_splitter.split_documents(docs)

print(f"Split the docs into {len(splits)} chunks.")

[Document(metadata={'producer': 'Adobe Acrobat Pro 2020 20 Paper Capture Plug-in', 'creator': 'Workiva', 'creationdate': '2024-03-27T20:21:55+00:00', 'author': 'JP Morgan Chase & Co.', 'keywords': 'Powering; Growth; with; Curiosity; and; Heart; Annual; Report; 2023; JP; Morgan; Chase; &; Co', 'moddate': '2024-09-23T08:04:47-04:00', 'subject': 'Annual Report 2023', 'title': 'Powering Growth with Curiosity and Heart Annual Report 2023', 'source': '/content/annualreport-2023.pdf', 'total_pages': 364, 'page': 0, 'page_label': ''}, page_content='Powering \nGrowth \nwith Curiosity  \nand Heart\nAnnual  \nReport  \n2023'), Document(metadata={'producer': 'Adobe Acrobat Pro 2020 20 Paper Capture Plug-in', 'creator': 'Workiva', 'creationdate': '2024-03-27T20:21:55+00:00', 'author': 'JP Morgan Chase & Co.', 'keywords': 'Powering; Growth; with; Curiosity; and; Heart; Annual; Report; 2023; JP; Morgan; Chase; &; Co', 'moddate': '2024-09-23T08:04:47-04:00', 'subject': 'Annual Report 2023', 'title': '

In [25]:
# Each chunk is also a document
splits[0]

Document(metadata={'producer': 'Adobe Acrobat Pro 2020 20 Paper Capture Plug-in', 'creator': 'Workiva', 'creationdate': '2024-03-27T20:21:55+00:00', 'author': 'JP Morgan Chase & Co.', 'keywords': 'Powering; Growth; with; Curiosity; and; Heart; Annual; Report; 2023; JP; Morgan; Chase; &; Co', 'moddate': '2024-09-23T08:04:47-04:00', 'subject': 'Annual Report 2023', 'title': 'Powering Growth with Curiosity and Heart Annual Report 2023', 'source': '/content/annualreport-2023.pdf', 'total_pages': 364, 'page': 0, 'page_label': '', 'start_index': 0}, page_content='Powering \nGrowth \nwith Curiosity  \nand Heart\nAnnual  \nReport  \n2023')

In [26]:
splits[0].metadata

{'producer': 'Adobe Acrobat Pro 2020 20 Paper Capture Plug-in',
 'creator': 'Workiva',
 'creationdate': '2024-03-27T20:21:55+00:00',
 'author': 'JP Morgan Chase & Co.',
 'keywords': 'Powering; Growth; with; Curiosity; and; Heart; Annual; Report; 2023; JP; Morgan; Chase; &; Co',
 'moddate': '2024-09-23T08:04:47-04:00',
 'subject': 'Annual Report 2023',
 'title': 'Powering Growth with Curiosity and Heart Annual Report 2023',
 'source': '/content/annualreport-2023.pdf',
 'total_pages': 364,
 'page': 0,
 'page_label': '',
 'start_index': 0}

In [27]:
splits[0].page_content

'Powering \nGrowth \nwith Curiosity  \nand Heart\nAnnual  \nReport  \n2023'

In [28]:
# Loading Documents
def load_documents(folder_path: str) -> List[Document]:
  documents = []
  for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
      file_path = os.path.join(folder_path, filename)
      loader = PyPDFLoader(file_path)
      documents.extend(loader.load())
  return documents

folder_path = "/content"
documents = load_documents(folder_path)

print(f"Loaded {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the docs into {len(splits)} chunks.")

Loaded 452 documents from the folder.
Split the docs into 2290 chunks.


In [ ]:
# Embedding Documents
# Explicitly pass the API key to the embeddings constructor
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", api_key="GOOGLE_API_KEY")
3
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])

print(f"Created {len(document_embeddings)} embeddings from documents.")

Created 2290 embeddings from documents.


In [30]:
document_embeddings[0]

[0.054351504892110825,
 0.016769271343946457,
 -0.0785428136587143,
 0.009664387442171574,
 0.08070209622383118,
 0.027904408052563667,
 0.004352941643446684,
 -0.035902220755815506,
 0.012247470207512379,
 0.05862659588456154,
 -0.014725386165082455,
 0.025941533967852592,
 -0.019030442461371422,
 0.041266102343797684,
 0.030402742326259613,
 -0.02754068188369274,
 0.033484429121017456,
 0.02591308020055294,
 0.018440887331962585,
 0.006959928665310144,
 -0.016006959602236748,
 0.0313270203769207,
 -0.011268818750977516,
 0.024733131751418114,
 0.020543595775961876,
 -0.026694029569625854,
 0.0033800755627453327,
 -0.05404609441757202,
 -0.02077673189342022,
 0.011973836459219456,
 -0.047406576573848724,
 0.02378656342625618,
 -0.048387862741947174,
 0.036521680653095245,
 -0.0019245378207415342,
 -0.03914317861199379,
 -0.009819363243877888,
 0.05236392468214035,
 0.004739356692880392,
 -0.006468036212027073,
 0.000890652067027986,
 -0.05196824297308922,
 -0.04147209972143173,
 0.005

In [31]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])

document_embeddings[0]

<ipython-input-31-b0ebd77f2c7c>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dataset

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[0.029285239055752754,
 0.045758962631225586,
 0.010355443693697453,
 0.08409811556339264,
 -0.03280055150389671,
 0.0052756620571017265,
 -0.01989138498902321,
 -0.021349571645259857,
 -0.047067027539014816,
 0.03582395240664482,
 0.0250408798456192,
 0.03527801111340523,
 -0.031719569116830826,
 0.0008272322011180222,
 -0.008520152419805527,
 0.08138801902532578,
 -0.018545877188444138,
 -0.033424295485019684,
 -0.08418112248182297,
 0.03359486907720566,
 0.0030492120422422886,
 0.04369377717375755,
 0.05818618834018707,
 0.05192577466368675,
 -0.017617247998714447,
 -0.014906788244843483,
 -0.0624297633767128,
 -0.012440415099263191,
 0.01975908875465393,
 0.0005976551910862327,
 0.018304985016584396,
 0.03727899491786957,
 0.12022252380847931,
 -0.00710223987698555,
 -0.040465351194143295,
 0.07302705198526382,
 0.06212663650512695,
 0.013542832806706429,
 -0.04548756405711174,
 0.01960812881588936,
 0.023471452295780182,
 -0.11677459627389908,
 0.03720895200967789,
 -0.00320489658

In [ ]:
from langchain_chroma import Chroma

embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key="GOOGLE_API_KEY")

collection_name = "my_collection"
vectorstore = Chroma.from_documents(collection_name=collection_name, documents=splits, embedding=embedding_function)
# Persist saves the database to the disk and data will be accessible even after the current program termination
# db.persist()

print("Vector store created and persisted to './chromadb'")

Vector store created and persisted to './chromadb'


In [33]:
# Performing Similarity Search

query = "Tell about JPMC's Finance in brief."
search_results = vectorstore.similarity_search(query)
print(len(search_results))

print(f"Top 2 relevant chunks for the query : '{query}' are :")
for i, result in enumerate(search_results, 1):
  print(f"Result {i}:")
  print(f"Source: {result.metadata.get('source', 'Unknown')}")
  print(f"Content: {result.page_content}")
  print()

4
Top 2 relevant chunks for the query : 'Tell about JPMC's Finance in brief.' are :
Result 1:
Source: /content/annualreport-2023.pdf
Content: and Hong Kong SAR funds and at the fund level for all other funds. 
The performance data may have been different if all share classes had 
been included. Past performance is not indicative of future results.
“Primary share class” means the C share class for European funds and 
Acc share class for Hong Kong SAR and Taiwan funds. If these share 
classes are not available, the oldest share class is used as the primary 
share class.
Selected metrics
As of or for the year ended 
December 31, 
(in millions, except ranking 
data, ratios and employees) 2023 2022 2021
% of JPM mutual fund assets 
and ETFs rated as 4- or 5-
star(a)  69 %  73 %  69 %
% of JPM mutual fund assets 
and ETFs ranked in 1st or 2nd 
quartile:(b)
1 year  40  68  54 
3 years  67  76  73 
5 years  71  81  80 
Selected balance sheet data 
(period-end)(c)
Total assets $ 245,512 $ 232,0

In [34]:
# Retriever object to get all the relevant docs from vector database
retriever = vectorstore.as_retriever(search_kwargs={"k":2})
retriever.invoke("Tell about JPMC's Finance in brief.")

[Document(id='7e8fdde4-b257-4d77-ba6b-6ec75dc9b6c2', metadata={'page_label': '82', 'author': 'JP Morgan Chase & Co.', 'start_index': 2488, 'source': '/content/annualreport-2023.pdf', 'total_pages': 364, 'moddate': '2024-09-23T08:04:47-04:00', 'page': 119, 'creationdate': '2024-03-27T20:21:55+00:00', 'subject': 'Annual Report 2023', 'producer': 'Adobe Acrobat Pro 2020 20 Paper Capture Plug-in', 'creator': 'Workiva', 'title': 'Powering Growth with Curiosity and Heart Annual Report 2023', 'keywords': 'Powering; Growth; with; Curiosity; and; Heart; Annual; Report; 2023; JP; Morgan; Chase; &; Co'}, page_content='and Hong Kong SAR funds and at the fund level for all other funds. \nThe performance data may have been different if all share classes had \nbeen included. Past performance is not indicative of future results.\n“Primary share class” means the C share class for European funds and \nAcc share class for Hong Kong SAR and Taiwan funds. If these share \nclasses are not available, the old

In [35]:
# Creating a template with proper prompts
template = """Answer the question based only on the following context:
{context}
Question: {Question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)

In [36]:
# from langchain.schema.runnable import RunnablePassthrough # Old import
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough forwards the input unchanged in pipelines that require no preprocessing.
# If used in a dictionary-based pipeline, it passes the input as is, or adds extra fields when needed.
rag_chain = (
    {"context": retriever, "Question": RunnablePassthrough()}
    | prompt
)

rag_chain.invoke("Tell about JPMC.")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(id='f6cc32ce-3c77-4beb-b050-46f4afe14dc3', metadata={'author': 'JPMorgan Chase & Co.', 'producer': 'Adobe PDF Library 17.0', 'creationdate': '2024-04-11T16:33:27-04:00', 'creator': 'Adobe InDesign 19.3 (Macintosh)', 'title': '2023 Environmental Social Governance Report', 'keywords': '2023, Environmental, Social, Governance, Report, JPMorgan, Chase, &, Co.', 'page': 70, 'total_pages': 88, 'source': '/content/jpmc-esg-report-2023.pdf', 'moddate': '2024-04-29T16:35:52-04:00', 'page_label': '71', 'subject': '2023 Environmental Social Governance Report', 'start_index': 2378, 'trapped': '/False'}, page_content='partner. Globally, we also enhanced parental leave to 16 weeks for parents \\nwho have or adopt a child. \\nWe also provide family-building assistance to help our U.S.-based employees \\nwith the high costs of adoption, surrogacy and fertility expenses, including up \\nt

In [37]:
def docs2str(docs):
  """
  Utility function to convert retrivered docs to proper string format.
  """
  return "\n\n".join(doc.page_content for doc in docs)

In [38]:
# RAG Chain with proper formatted docs being put into the prompt.
# This chain has does process only till the prompt templating - return the prompt with retrieved docs formatted and inserted into them properly.
rag_chain = (
    {"context": retriever | docs2str, "Question": RunnablePassthrough()}
    | prompt
)

rag_chain.invoke("Tell me about JPMC.")

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\npartner. Globally, we also enhanced parental leave to 16 weeks for parents \nwho have or adopt a child. \nWe also provide family-building assistance to help our U.S.-based employees \nwith the high costs of adoption, surrogacy and fertility expenses, including up \nto $10,000 per child in eligible adoption expenses, up to a $30,000 lifetime \nmaximum for surrogacy and $40,000 lifetime maximum for eligible \nfertility expenses. \nOur Parents@jpmc program ofers educational sessions with parenting experts \nand connects parents with a wide range of other programs ofered by our \nFirm. For those with young children, we try to remove some of the stress \nassociated with fnding and paying for childcare by ofering access to \ncomprehensive childcare support. This includes access to on-site and near-site \nchildcare centers for full-service and back-up care needs in many large \nlocations.

In [39]:
# This rag chain invokes the llm and parses output and displays only the page_content from the json returned
rag_chain = (
    {"context":retriever | docs2str, "Question": RunnablePassthrough()}
    | prompt
    | google_llm
    | output_parser
)

from pprint import pprint

# rag_chain.invoke("Tell me about JPMC")
# question = "When was JPMC founded?"
question = "Tell me about JPMC."
# question = "What does JPMC do?"
response = rag_chain.invoke(question)
pprint(response)

('JPMorgan Chase (JPMC) is now the #1 bank by market capitalization. Each of '
 'its businesses is among the best in the world, with increased market share, '
 'strong financial results and an unwavering focus on serving its clients, '
 'communities and shareholders. Its strengths include the knowledge and '
 'cohesiveness of its people, long-standing client relationships, technology '
 'and product capabilities, a presence in more than 100 countries, and a '
 'strong balance sheet. The company provides 16 weeks of parental leave. It '
 'also provides family-building assistance to U.S.-based employees, including '
 'up to $10,000 per child in eligible adoption expenses, up to a $30,000 '
 'lifetime maximum for surrogacy and $40,000 lifetime maximum for eligible '
 'fertility expenses. They offer a Parents@jpmc program and comprehensive '
 'childcare support.')


# **Conversational RAG**
Handles follow-up questions

In [40]:
from langchain_core.messages import HumanMessage, AIMessage

# To enable conversational RAG, chat_history has to be maintained.
# This list stores Human message and AI message in a list for future use.
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

chat_history

[HumanMessage(content='Tell me about JPMC.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='JPMorgan Chase (JPMC) is now the #1 bank by market capitalization. Each of its businesses is among the best in the world, with increased market share, strong financial results and an unwavering focus on serving its clients, communities and shareholders. Its strengths include the knowledge and cohesiveness of its people, long-standing client relationships, technology and product capabilities, a presence in more than 100 countries, and a strong balance sheet. The company provides 16 weeks of parental leave. It also provides family-building assistance to U.S.-based employees, including up to $10,000 per child in eligible adoption expenses, up to a $30,000 lifetime maximum for surrogacy and $40,000 lifetime maximum for eligible fertility expenses. They offer a Parents@jpmc program and comprehensive childcare support.', additional_kwargs={}, response_metadata={})]

In [41]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

# Prompt to give answers based on the context and chat history given

# contextualise_q_system_prompt = (
#     "Given a chat history and the latest user question"
#     "which might reference context in the chat history, "
#     "formulate a standalone question which can be understood"
#     "without the chat history. Do NOT answer the question, "
#     "just reformulate it if  needed and otherwise return it as it is"
# )

contextualise_q_system_prompt = (
    "Rephrase the given question and Answer the given question based on the given context and history. Display the rephrased question along with the answer"
)

# Creating a template to insert into the retrieval chain.
contextualise_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualise_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# Chain built on the contextualised prompt given and used in this chain
contextualise_chain = contextualise_q_prompt | google_llm | output_parser
contextualise_chain.invoke({"input": "Where is this company available?",
                            "chat_history": chat_history})

'**Rephrased Question:** In which locations does JPMorgan Chase operate?\n\n**Answer:** JPMorgan Chase has a presence in more than 100 countries.'

In [42]:
from langchain.chains import create_history_aware_retriever

# History-aware retriever: Enhances document retrieval by considering past chat history.
# It retrieves relevant documents based on the input query while incorporating conversation context.
history_aware_retriever = create_history_aware_retriever(
    google_llm, retriever, contextualise_q_prompt
)

history_aware_retriever.invoke({"input":"Where is this company available?",
                                "chat_history": chat_history})

[Document(id='57bccea8-6f2f-420e-a11c-2a8d9aba01ef', metadata={'author': 'JPMorgan Chase & Co.', 'moddate': '2024-04-29T16:35:52-04:00', 'producer': 'Adobe PDF Library 17.0', 'total_pages': 88, 'creationdate': '2024-04-11T16:33:27-04:00', 'start_index': 0, 'creator': 'Adobe InDesign 19.3 (Macintosh)', 'subject': '2023 Environmental Social Governance Report', 'trapped': '/False', 'page': 5, 'title': '2023 Environmental Social Governance Report', 'keywords': '2023, Environmental, Social, Governance, Report, JPMorgan, Chase, &, Co.', 'source': '/content/jpmc-esg-report-2023.pdf', 'page_label': '6'}, page_content='Operational excellence \n \nA g\nreat team and winning \nculture \nCompany at a Glance \nINTRODUCTION \nMessage from Our Chairman \nCompany at a Glance \nOur Approach to ESG \nOur $2.5T Sustainable \nDevelopment Target \nGOVERNANCE \nENVIRONMENTAL \nSOCIAL \nAPPENDICES \nHow We Do Business \nJPMorgan Chase & Co. (“JPMorgan Chase”, the “Firm” or \n“we”) is a fnancial services frm 

In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# RAG pipeline setup:
# - `qa_prompt`: Defines how the assistant answers using retrieved context and chat history.
# - `question_answer_chain`: Combines retrieved documents to generate responses.
# - `rag_chain`: Links retrieval with the response generator, ensuring context-aware answers.
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

# `create_stuff_documents_chain`: Merges retrieved documents into a structured prompt.
# Ensures all relevant content is formatted before passing it to the LLM for response generation.
# This step improves contextual understanding by consolidating multiple sources into a single coherent input.
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# `create_retrieval_chain`: Retrieves relevant documents based on the query
# and passes them to the response generation chain for context-aware answers.
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)



"""A normal retriever (like a vector store retriever) does not inherently support conversational retrieval because
 it only fetches documents based on the current query—it does not consider past interactions.
 This function - create_retrieval_chain supports conversational retrieval on top of the standard retrieval.
 It takes a retriever and chain made out of a prompt template.
"""

'A normal retriever (like a vector store retriever) does not inherently support conversational retrieval because\n it only fetches documents based on the current query—it does not consider past interactions.\n This function - create_retrieval_chain supports conversational retrieval on top of the standard retrieval.\n It takes a retriever and chain made out of a prompt template.\n'

In [44]:
# Invoking the created retrieval chain
result = rag_chain.invoke({"input":"Where are the company's services available?",
                  "chat_history": chat_history})

pprint(result)

{'answer': 'JPMorgan Chase & Co. has branches in 48 states and Washington '
           'D.C., 309,926 employees in 65 countries. The Firm serves millions '
           'of customers, predominantly in the U.S., and many of the world’s '
           'most prominent corporate, institutional and government clients '
           'globally. JPMorgan Chase’s activities involve clients residing '
           'outside of the U.S.',
 'chat_history': [HumanMessage(content='Tell me about JPMC.', additional_kwargs={}, response_metadata={}),
                  AIMessage(content='JPMorgan Chase (JPMC) is now the #1 bank by market capitalization. Each of its businesses is among the best in the world, with increased market share, strong financial results and an unwavering focus on serving its clients, communities and shareholders. Its strengths include the knowledge and cohesiveness of its people, long-standing client relationships, technology and product capabilities, a presence in more than 100 countries,

# **Building a Multi User Chatbot**
Using Sqlite3 database

In [45]:
import sqlite3
from datetime import datetime

DB_NAME = "rag_app.db"

def get_db_connection():
  conn = sqlite3.connect(DB_NAME)
  conn.row_factory = sqlite3.Row
  return conn

def create_application_logs():
  conn = get_db_connection()
  cursor = conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
                        (id INTEGER PRIMARY KEY AUTOINCREMENT,
                        session_id TEXT,
                        user_query TEXT,
                        llm_response TEXT,
                        model TEXT,
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)
                        ''')
  conn.close()

def insert_application_logs(session_id, user_query, llm_response, model):
  conn = get_db_connection()
  conn.execute('INSERT INTO application_logs (session_id, user_query, llm_response, model) VALUES (?, ?, ?, ?)',
               (session_id, user_query, llm_response, model))
  conn.commit()
  conn.close()

def get_chat_history(session_id):
  conn = get_db_connection()
  cursor = conn.execute('SELECT * FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
  messages = []
  for row in cursor.fetchall():
    messages.extend([
        {"role":"human", "content": row['user_query']},
        {"role":"ai", "content": row['llm_response']}
    ])
  conn.close()
  return messages

# Initialize the database
create_application_logs()

In [46]:
# USER 1

import uuid
session_id = str(uuid.uuid4())

chat_history = get_chat_history(session_id)
print(chat_history)

question1 = "Where is this company available?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question1, answer1, google_llm.model)

chat_history.extend([
    {"role":"human", "content": question1},
    {"role":"ai", "content": answer1}
])
pprint(chat_history[-1])

[]
{'content': 'JPMorgan Chase & Co. is a financial services firm based in the '
            'United States of America, with branches in 48 states and '
            'Washington D.C. The firm has 309,926 employees in 65 countries.',
 'role': 'ai'}


In [47]:
pprint(chat_history)

[{'content': 'Where is this company available?', 'role': 'human'},
 {'content': 'JPMorgan Chase & Co. is a financial services firm based in the '
             'United States of America, with branches in 48 states and '
             'Washington D.C. The firm has 309,926 employees in 65 countries.',
  'role': 'ai'}]


In [48]:
question2 = "What does it do?"
# Using same session id considers chat history while answering a new question
chat_history = get_chat_history(session_id)
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question2, answer2, google_llm.model)

In [49]:
pprint(answer2)

('JPMorgan Chase & Co. provides financial services for individuals and '
 'industries across geographies. They serve more than 82 million consumer '
 'customers in the U.S., 6.4 million small businesses, and hundreds of '
 'thousands of companies in critical economic sectors. The firm operates '
 'through four major reportable business segments: Consumer & Community '
 'Banking, Corporate & Investment Bank, Commercial Banking, and Asset & Wealth '
 'Management, with an additional Corporate segment. Consumer & Community '
 'Banking offers products and services to consumers and small businesses '
 'through bank.')


In [50]:
chat_history = get_chat_history(session_id)
pprint(chat_history)

[{'content': 'Where is this company available?', 'role': 'human'},
 {'content': 'JPMorgan Chase & Co. is a financial services firm based in the '
             'United States of America, with branches in 48 states and '
             'Washington D.C. The firm has 309,926 employees in 65 countries.',
  'role': 'ai'},
 {'content': 'What does it do?', 'role': 'human'},
 {'content': 'JPMorgan Chase & Co. provides financial services for individuals '
             'and industries across geographies. They serve more than 82 '
             'million consumer customers in the U.S., 6.4 million small '
             'businesses, and hundreds of thousands of companies in critical '
             'economic sectors. The firm operates through four major '
             'reportable business segments: Consumer & Community Banking, '
             'Corporate & Investment Bank, Commercial Banking, and Asset & '
             'Wealth Management, with an additional Corporate segment. '
             'Consumer & Co

**New User**

In [52]:
# Generating a new session key for new user
session_id = str(uuid.uuid4())

chat_history = get_chat_history(session_id)
print(chat_history)

question = "When was JPMC founded?"
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question, answer, google_llm.model)

chat_history.extend([
    {"role":"human", "content": question},
    {"role": "ai", "content": answer}
])

pprint(chat_history)

[]
[{'content': 'When was JPMC founded?', 'role': 'human'},
 {'content': 'I am sorry, but this document does not contain information about '
             'when JPMC was founded. However, it does mention some recent '
             'events, such as branch openings and key executive changes.',
  'role': 'ai'}]
